In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import torch
from sklearn.decomposition import PCA
from tqdm import tqdm
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import numpy as np
import seaborn as sns
import os
from chemUtils.utils import utils
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from matplotlib.lines import Line2D
import plotly.express as px
import json

In [2]:
data = pd.read_csv('figure_S2.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'figure_S2.csv'

In [ ]:
data

In [ ]:
data.iloc[8]

In [ ]:
def extract_rxn_conditions(old_df, df):
    # Extract metadata from the 0,0 entry
    print(df)
    if old_df is None:
        metadata = df.columns[0].split(", ")
    else:
        metadata = df.iloc[0,0].split(", ")
    print(metadata)
    T = metadata[0][:-1]
    time = metadata[1].split(" ")[0][:-1]
    base, solvent = metadata[2:4]
    round_num = metadata[4][1]
    
    # Create an empty DataFrame with the desired columns
    new_df = pd.DataFrame(columns=['A', 'B', 'solvent', 'base', 'T', 'time', 'round_num', 'peak_height'])
    
    # Populate the new DataFrame
    for col in df.columns[1:]:
        for index, row in df.iterrows():
            if "C" in row[0]:
                continue
            new_row = {
                'A': row[0],
                'B': col,
                'solvent': solvent,
                'base': base,
                'T': T,
                'time': time,
                'round_num': round_num,
                'peak_height': row[col]
            }
            new_df = new_df.append(new_row, ignore_index=True)
    
    if old_df is None:
        return new_df
    else:
        return old_df.append(new_df, ignore_index=True)
    

In [ ]:
new_df = None
new_df = extract_rxn_conditions(None, data.iloc[:8])
for i in range(1, 16):
    print(i)
    new_df = extract_rxn_conditions(new_df, data.iloc[(i*9)-1:(i+1)*9-1])

In [ ]:
new_df

In [ ]:
new_df.to_csv('figure_S2_tidy.csv', index=False)

# Making 5 random train and test splits of 80/20

In [ ]:
os.chdir('/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/')
df = pd.read_csv('figure_S2_tidy.csv')
df

In [ ]:
num_splits = 5
train_test_splits = []

for _ in range(num_splits):
    # Randomly shuffle the indices
    shuffled_indices = torch.randperm(len(df))
    
    # Split indices into train and test
    split_idx = int(0.8 * len(df))
    train_indices = shuffled_indices[:split_idx]
    test_indices = shuffled_indices[split_idx:]
    
    # Extract train and test DataFrames based on indices
    train_df = df.iloc[train_indices]
    test_df = df.iloc[test_indices]
    
    train_test_splits.append((train_df, test_df))

In [ ]:
for i, (train, test) in enumerate(train_test_splits):
    print(f"Split {i+1}")
    print("Train:", len(train))
    print("Test:", len(test))
    print("------")

In [ ]:
for i, (train, test) in enumerate(train_test_splits):
    train.to_csv(f'figure_S2_train_{i+1}.csv', index=False)
    test.to_csv(f'figure_S2_test_{i+1}.csv', index=False)

# Add smiles for each reactant.

In [ ]:
smiles = {
    'R1-A1': 'N[C@@H]1CCCC[C@H]1N',
    'R1-A2': 'Cn1ccnc1',
    'R1-A3': 'C1CCNCC1',
    'R1-A4': 'CN1CCNCC1',
    'R1-A5': 'NCc1ccccc1',
    'R1-A6': 'Nc1cccnc1N',
    'R1-A7': 'CC1CCCCN1CCCN',
    'R1-A8': 'c1ccc2[nH]cnc2c1',
    'R1-B1': 'CC(=O)c1ccc(F)cc1',
    'R1-B2': 'CC(=O)c1ccc(Cl)cc1',
    'R1-B3': 'CC(=O)c1ccc(Br)cc1',
    'R1-B4': 'O=[N+]([O-])c1ccc(F)cc1',
    'R1-B5': 'O=[N+]([O-])c1ccc(Cl)cc1',
    'R1-B6': 'O=[N+]([O-])c1ccc(Br)cc1',
    'R1-B7': 'CCc1ncnc(Cl)c1F',
    'R1-B8': 'Fc1ccc(C(F)(F)F)cc1',
    'R1-B9': 'FC(F)(F)c1ccc(Cl)cc1',
    'R1-B10': 'N#Cc1ccc(Br)cc1',
    'R1-B11': 'CCN(CC)c1ccc(Br)cc1',
    'R1-B12 (S)': 'O=[N+]([O-])c1cnc(Cl)nc1Cl',
    'R1-B12 (D)': 'O=[N+]([O-])c1cnc(Cl)nc1Cl',
}

In [ ]:
def is_canonical(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    canonical_smiles = Chem.MolToSmiles(molecule)
    return smiles == canonical_smiles

smiles_str = "CC(=O)c1ccc(F)cc1"   # example SMILES string
print(is_canonical(smiles_str))

In [ ]:
# Add smiles as column to dataframes
for i in range(1, 6):
    train_df = pd.read_csv(f'figure_S2_train_{i}.csv')
    test_df = pd.read_csv(f'figure_S2_test_{i}.csv')
    
    train_df['A_smiles'] = train_df['A'].apply(lambda x: smiles[x])
    train_df['B_smiles'] = train_df['B'].apply(lambda x: smiles[x])
    test_df['A_smiles'] = test_df['A'].apply(lambda x: smiles[x])
    test_df['B_smiles'] = test_df['B'].apply(lambda x: smiles[x])
    
    test_df = test_df[['A', 'A_smiles', 'B', 'B_smiles', 'solvent', 'base', 'T', 'time', 'round_num', 'peak_height']]
    train_df = train_df[['A', 'A_smiles', 'B', 'B_smiles', 'solvent', 'base', 'T', 'time', 'round_num', 'peak_height']]
    
    train_df.to_csv(f'figure_S2_train_{i}.csv', index=False)
    test_df.to_csv(f'figure_S2_test_{i}.csv', index=False)

In [ ]:
# Make fp list
def fp_as_array(mol,n_bits=2048):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits)
    arr = np.zeros((1,), int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

def fp_list_from_smiles_list(smiles_list,n_bits=2048):
    fp_list = []
    for smiles in tqdm(smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        fp_list.append(fp_as_array(mol,n_bits))
    return fp_list

fp_list = fp_list_from_smiles_list(list(smiles.values()))

In [ ]:
# show T-SNE plot of amines and aryl halides, using first 50 PCs
sns.set(rc={'figure.figsize': (10, 10)})
sns.set(font_scale=1.5)
sns.set_style('whitegrid')

pca = PCA(n_components=50)
crds = pca.fit_transform(fp_list) 
from sklearn.manifold import TSNE
%time crds_embedded = TSNE(n_components=2).fit_transform(crds)

# Add other representations of peak height data

In [ ]:
os.chdir('/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr')
df = pd.read_csv('figure_S2_tidy.csv')
df['A_smiles'] = df['A'].apply(lambda x: smiles[x])
df['B_smiles'] = df['B'].apply(lambda x: smiles[x])
df = df[['A', 'A_smiles', 'B', 'B_smiles', 'solvent', 'base', 'T', 'time', 'round_num', 'peak_height']]
df = df[df['B'].isin(['R1-B12 (S)']) == False]
df.to_csv(f'figure_S2_tidy.csv', index=False)

In [ ]:
sns.set(rc={'figure.figsize': (10, 10)})
sns.set(font_scale=1.5)
sns.set_style('whitegrid')
# What is the distribution of peak heights?
sns.histplot(data=df, x='peak_height', bins=20, kde=True)

In [ ]:
# Min-max scaling
df['peak_height_scaled'] = (df['peak_height'] - df['peak_height'].min()) / (df['peak_height'].max() - df['peak_height'].min())
sns.histplot(data=df, x='peak_height_scaled', bins=20, kde=True)

In [ ]:
df['peak_height_scaled'].describe()

In [ ]:
df['peak_height_log'] = np.log(df['peak_height'])
sns.histplot(data=df, x='peak_height_log', bins=20, kde=True)

In [ ]:
df.to_csv(f'figure_S2_tidy.csv', index=False)
df

In [ ]:
# Assuming df is your DataFrame and 'Original Data' is the column to be scaled
data = torch.tensor(df['peak_height'].values, dtype=torch.float32)

# Sort the data and get the indices
sorted_data, indices = torch.sort(data)

# Compute the quantile rank for each value
quantile_rank = torch.arange(1, len(data) + 1, dtype=torch.float32) / (len(data) + 1)

# Map the data to its quantile rank
quantile_scaled_data = torch.zeros_like(data)
quantile_scaled_data[indices] = quantile_rank

# Add the quantile-scaled data as a new column in the original DataFrame
df['peak_height_quantile_scaled'] = quantile_scaled_data.numpy()

df

In [ ]:
df.drop(columns=['Quantile Scaled Data'], inplace=True)

In [ ]:
df

In [ ]:
quantile_labels = ['Q1', 'Q2', 'Q3', 'Q4']
df['quantile_category'] = pd.qcut(df['peak_height'], q=4, labels=quantile_labels)
df

In [ ]:
sns.set(rc={'figure.figsize': (10, 10)})
sns.set(font_scale=1.5)
sns.set_style('whitegrid')
sns.scatterplot(data=df, x='peak_height_quantile_scaled', y='peak_height', hue='quantile_category')

In [ ]:
decile_labels = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']
df['decile_category'] = pd.qcut(df['peak_height'], q=10, labels=decile_labels)
sns.scatterplot(data=df, x='peak_height_quantile_scaled', y='peak_height', hue='decile_category')

In [ ]:
df.to_csv(f'figure_S2_tidy.csv', index=False)

# TEST TSNE

In [ ]:
# Make images dictionary
filename = '/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2/benchmark_init_random_embedding_OHE_scaling_log_batch_5_acq_EHVI.csv'
df_preds = pd.read_csv(filename)
A = df_preds['A'].values
# get unique values in A
A_unique = np.unique(A)
A_unique

B_unique = np.unique(df_preds['B'].values)
full = np.concatenate((A_unique, B_unique))

images = {}
for label in full:
    images[label] = f'{label}.png'
with open('/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2/images.json', 'w') as f:
    json.dump(images, f)

In [ ]:
def _plot_batches(embedding=False, concat=True, image_file_path=None, image_dict=None):
    """
    Plot the batches in a T-SNE plot.
    
    embedding: bool
        If True, it will plot the full embedding of the batches. Else, will just plot fp of reactants.
    concat: bool
        If True, it will plot the fp of reactants concatenated. Else, will plot them separately.
    image_file_path: str
        Path to the folder of images files in dictionary.
    image_dict: dict
        Dictionary of images and their filenames.
    """
    filename = '/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2/benchmark_init_random_embedding_OHE_scaling_log_batch_5_acq_EHVI.csv'
    
    df_preds = pd.read_csv(filename)
    r1_test = df_preds[df_preds['batch_num'].notna()]['A_smiles'].values
    r2_test = df_preds[df_preds['batch_num'].notna()]['B_smiles'].values

    # Get fp list (or lists)
    if concat:
        reacts_fp = utils.fp_list_from_two_smiles_list(r1_test, r2_test)
    else:
        react1_fp = utils.fp_list_from_smiles_list(r1_test)
        react2_fp = utils.fp_list_from_smiles_list(r2_test)
    pca = PCA(n_components=10)
    # Get coordinates and plot
    plt.figure(figsize=(10, 8))
    if concat:
        crds = pca.fit_transform(reacts_fp)
        crds_tsne = TSNE(n_components=2).fit_transform(crds)
        reacts_df = pd.DataFrame(crds_tsne, columns=["X", "Y"])
        reacts_df['batch_num'] = df_preds[df_preds['batch_num'].notna()]['batch_num'].values
        reacts_df['react1_smiles'] = r1_test
        reacts_df['react2_smiles'] = r2_test
        
        # Compute average X and Y coordinates for each unique react1 and react2 smiles combo.
        averaged_coords = reacts_df.groupby(['react1_smiles', 'react2_smiles']).agg({'X': 'mean', 'Y': 'mean'}).reset_index()
        # Merge the averaged coordinates back to the original dataframe
        plot_df = reacts_df.drop(columns=['X', 'Y']).merge(averaged_coords, on=['react1_smiles', 'react2_smiles'], how='left')
        
        # Plot
        plot_df['batch_num'] = plot_df['batch_num'].astype(str)
        fig = px.scatter(plot_df, x="X", y="Y", color="batch_num", hover_data=['react1_smiles', 'react2_smiles'])
        fig.update_traces(marker={'size': 15})
        fig.write_html("test_scatter.html")
    else:
        return NotImplementedError
    fig.show()
    return plot_df

In [ ]:
with open('/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2/images.json', 'r') as f:
    images = json.load(f)
_plot_batches(image_dict=images, concat=True, image_file_path='/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2/')

# Make csv of highest peak heights for each reaction.

In [4]:
os.chdir('/Users/kate_fieseler/PycharmProjects/edboplus_scope/test/thompson_SNAr/figure_S2/data')
df = pd.read_csv('figure_S2_tidy.csv')
df

,A,A_smiles,B,B_smiles,solvent,base,T,time,round_num,peak_height,peak_height_scaled,peak_height_log,peak_height_quantile_scaled,quantile_category,peak_height_decile
0,R1-A1,N[C@@H]1CCCC[C@H]1N,R1-B1,CC(=O)c1ccc(F)cc1,NMP,DIPEA,20,0,1,31.7,0.001923,3.456317,0.303839,Q2,D4
1,R1-A2,Cn1ccnc1,R1-B1,CC(=O)c1ccc(F)cc1,NMP,DIPEA,20,0,1,52.3,0.003266,3.956996,0.482759,Q2,D5
2,R1-A3,C1CCNCC1,R1-B1,CC(=O)c1ccc(F)cc1,NMP,DIPEA,20,0,1,12.2,0.000652,2.501436,0.057254,Q1,D1
3,R1-A4,CN1CCNCC1,R1-B1,CC(=O)c1ccc(F)cc1,NMP,DIPEA,20,0,1,410.3,0.026602,6.016889,0.811321,Q4,D9
4,R1-A5,NCc1ccccc1,R1-B1,CC(=O)c1ccc(F)cc1,NMP,DIPEA,20,0,1,24.3,0.001441,3.190476,0.191282,Q1,D2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,R1-A4,CN1CCNCC1,R1-B12 (D),O=[N+]([O-])c1cnc(Cl)nc1Cl,"1,4-dioxane",None,150,15,1,50.0,0.003116,3.912023,0.469096,Q2,D5
1532,R1-A5,NCc1ccccc1,R1-B12 (D),O=[N+]([O-])c1cnc(Cl)nc1Cl,"1,4-dioxane",None,150,15,1,79.0,0.005006,4.369448,0.595966,Q3,D6
1533,R1-A6,Nc1cccnc1N,R1-B12 (D),O=[N+]([O-])c1cnc(Cl)nc1Cl,"1,4-dioxane",None,150,15,1,83.6,0.005306,4.426044,0.609629,Q3,D7
1534,R1-A7,CC1CCCCN1CCCN,R1-B12 (D),O=[N+]([O-])c1cnc(Cl)nc1Cl,"1,4-dioxane",None,150,15,1,103.2,0.006584,4.636669,0.649317,Q3,D7


In [7]:
# Group by the 'R1-AX' and 'R1-BX' and compute the max of 'peak_height'
grouped = df.groupby(['A', 'B'])['peak_height'].max().reset_index()

# Merge with the original DataFrame
merged = pd.merge(df, grouped, on=['A', 'B', 'peak_height'])

# Print the resulting DataFrame
merged

,A,A_smiles,B,B_smiles,solvent,base,T,time,round_num,peak_height,peak_height_scaled,peak_height_log,peak_height_quantile_scaled,quantile_category,peak_height_decile
0,R1-A4,CN1CCNCC1,R1-B12 (D),O=[N+]([O-])c1cnc(Cl)nc1Cl,NMP,DIPEA,20,0,1,2215.1,0.144249,7.703053,0.936890,Q4,D10
1,R1-A4,CN1CCNCC1,R1-B1,CC(=O)c1ccc(F)cc1,NMP,DIPEA,150,15,1,3499.9,0.228000,8.160490,0.956409,Q4,D10
2,R1-A7,CC1CCCCN1CCCN,R1-B3,CC(=O)c1ccc(Br)cc1,NMP,DIPEA,150,15,1,826.9,0.053759,6.717684,0.874431,Q4,D9
3,R1-A7,CC1CCCCN1CCCN,R1-B4,O=[N+]([O-])c1ccc(F)cc1,NMP,DIPEA,150,15,1,9886.8,0.644334,9.198956,0.993494,Q4,D10
4,R1-A1,N[C@@H]1CCCC[C@H]1N,R1-B5,O=[N+]([O-])c1ccc(Cl)cc1,NMP,DIPEA,150,15,1,1555.6,0.101259,7.349617,0.908914,Q4,D10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,R1-A6,Nc1cccnc1N,R1-B2,CC(=O)c1ccc(Cl)cc1,"1,4-dioxane",None,150,15,1,106.2,0.006779,4.665324,0.651269,Q3,D7
92,R1-A2,Cn1ccnc1,R1-B3,CC(=O)c1ccc(Br)cc1,"1,4-dioxane",None,150,15,1,255.8,0.016531,5.544396,0.770332,Q4,D8
93,R1-A6,Nc1cccnc1N,R1-B3,CC(=O)c1ccc(Br)cc1,"1,4-dioxane",None,150,15,1,109.7,0.007007,4.697749,0.658425,Q3,D7
94,R1-A5,NCc1ccccc1,R1-B7,CCc1ncnc(Cl)c1F,"1,4-dioxane",None,150,15,1,8215.1,0.535363,9.013729,0.987638,Q4,D10


In [8]:
merged.to_csv('figure_S2_tidy_max_peak_height.csv', index=False)